<a href="https://colab.research.google.com/github/mkkim007/AIFFEL/blob/main/%5BE_06%5DMyFirstKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [E-06]나의 첫 번째 캐글 경진대회


## 0.시작하기전,,

[캐글 코리아와 함께하는 2nd ML 대회 - House Price Prediction](https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr)

RMSE
이번 대회의 평가 방식은 Root Mean Squared Error 입니다.
$${\sqrt{ {1 \over N} \sum{(yt - y{pr})}^2}} $$

In [ ]:
!pip list | grep xgboost
!pip list | grep lightgbm
!pip list | grep missingno
!pip list | grep scikit-learn

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!wget https://d3s0tskafalll9.cloudfront.net/media/documents/kaggle-kakr-housing-data.zip

In [ ]:
!unzip -qq '/content/kaggle-kakr-housing-data.zip'

In [ ]:
ls /content/data

## 1.데이터 및 라이브러리 준비하기

###1.1필요한 라이브러리 import

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import norm #Analysis 
from scipy import stats #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings 
warnings.filterwarnings('ignore')

###1.2 데이터 불러오기

In [ ]:
data_dir = '/content/data'

train_data_path = join(data_dir, 'train.csv')   
sub_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

print(train_data_path)
print(sub_data_path)

In [ ]:
data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)
print('train data dim : {}'.format(data.shape))
print('sub data dim : {}'.format(sub.shape))

## 2.데이터 탐색 (EDA : Exploratory Data Analysis)

### 2.1 변수 

- ID : 집을 구분하는 번호  
- date : 집을 구매한 날짜
- price : 집의 가격(Target variable)
- bedrooms : 침실의 수
- bathrooms : 화장실의 수
- sqft_living : 주거 공간의 평방 피트(면적)
- sqft_lot : 부지의 평방 피트(면적)
- floors : 집의 층 수
- waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
- view : Has been viewed
- condition : 집의 전반적인 상태
- grade : King County grading 시스템 기준으로 매긴 집의 등급
- sqft_above : 지하실을 제외한 평방 피트(면적)
- sqft_basement : 지하실의 평방 피트(면적)
- yr_built : 지어진 년도
- yr_renovated : 집을 재건축한 년도
- zipcode : 우편번호
- lat : 위도
- long : 경도
- sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
- sqft_lot15 : 2015년 기준 주차 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

In [ ]:
data.head()

### 학습 데이터에서 라벨 제거하기

In [ ]:
y = data['price']
del data['price']

print(data.columns)

#### 1.2.3 학습데이터와 테스트 데이터 합치기

In [ ]:
train_len = len(data)
data = pd.concat((data, sub), axis=0)

print(len(data))

In [ ]:
data.head()